<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB; /* رنگ پس‌زمینه بلوک‌های کد */
            padding: 10px;
            border-radius: 5px;
            direction: ltr; /* جهت متنی کدها از چپ به راست */
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 14px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul, ol {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        ul ul, ul ol, ol ul, ol ol {
            padding-right: 20px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <h1>Retrieval-Augmented Generation</h1>
    <p>RAG یا <strong>"Retrieval-Augmented Generation"</strong> یک رویکرد نوین در پردازش زبان طبیعی (NLP) است که ترکیبی از دو تکنیک اصلی یعنی <strong>"بازیابی"</strong> (Retrieval) و <strong>"تولید"</strong> (Generation) را به کار می‌گیرد. این روش به ویژه در زمینه‌های مانند پاسخ‌گویی به سوالات، چت‌بات‌ها و تولید متن‌های اطلاعاتی کاربرد دارد.</p> 
    <h2>۱. اجزای RAG</h2>
    <ul>
        <li>
            <strong>بازیابی (Retrieval):</strong>
            <p>در این مرحله، سیستم به دنبال جستجو و بازیابی اطلاعات مرتبط از یک پایگاه داده یا مجموعه‌ای از متون می‌گردد. این اطلاعات می‌تواند شامل مقالات، وب‌سایت‌ها، یا هر نوع داده متنی دیگری باشد. هدف این است که اطلاعاتی که به سوال یا موضوع مطرح شده مرتبط است، شناسایی و استخراج شود.</p>
        </li>
        <li>
            <strong>تولید (Generation):</strong>
            <p>پس از بازیابی اطلاعات، سیستم از این داده‌ها برای تولید پاسخ یا متنی جدید استفاده می‌کند. این مرحله معمولاً با استفاده از مدل‌های تولید متن مانند GPT یا BERT انجام می‌شود. در اینجا، سیستم می‌تواند اطلاعات بازیابی شده را ترکیب کرده و پاسخی مناسب و منطقی تولید کند.</p>
        </li>
    </ul>
    <h2>۲. مزایای RAG</h2>
    <ul>
        <li><strong>دقت بیشتر:</strong> با بازیابی اطلاعات مرتبط، دقت پاسخ‌ها افزایش می‌یابد.</li>
        <li><strong>تنوع پاسخ‌ها:</strong> به دلیل استفاده از اطلاعات مختلف، پاسخ‌ها می‌توانند متنوع‌تر و جامع‌تر باشند.</li>
        <li><strong>کاهش بار محاسباتی:</strong> به جای اینکه تمام اطلاعات در مدل ذخیره شود، می‌توان از منابع خارجی برای بازیابی استفاده کرد.</li>
    </ul>
    <h2>۳. کاربردها</h2>
    <ul>
        <li><strong>پاسخ به سوالات:</strong> به ویژه در سیستم‌های پرسش و پاسخ که نیاز به اطلاعات دقیق و به‌روز دارند.</li>
        <li><strong>چت‌بات‌ها:</strong> برای بهبود کیفیت گفتگوها و ارائه پاسخ‌های دقیق‌تر.</li>
        <li><strong>تولید محتوا:</strong> برای نوشتن مقالات یا متون علمی با استناد به منابع معتبر.</li>
    </ul>
    <h2>۴. چالش‌ها</h2>
    <ul>
        <li><strong>کیفیت داده‌های بازیابی شده:</strong> اگر داده‌های بازیابی شده بی‌کیفیت یا نامرتبط باشند، پاسخ نهایی نیز تحت تأثیر قرار می‌گیرد.</li>
        <li><strong>زمان پاسخ‌دهی:</strong> ممکن است زمان بیشتری برای بازیابی اطلاعات نیاز باشد که می‌تواند تجربه کاربری را تحت تأثیر قرار دهد.</li>
    </ul>
    <p>به طور کلی، RAG یک رویکرد قدرتمند در پردازش زبان طبیعی است که می‌تواند به بهبود کیفیت و دقت پاسخ‌ها کمک کند.</p>
</body>
</html>


________

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>نصب Llama 3.1</title>
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB; /* رنگ پس‌زمینه بلوک‌های کد */
            padding: 10px;
            border-radius: 5px;
            direction: ltr; /* جهت متنی کدها از چپ به راست */
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 14px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul, ol {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        ul ul, ul ol, ol ul, ol ol {
            padding-right: 20px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
        img {
            max-width: 100%;
            height: auto;
            margin-bottom: 20px;
        }
    </style>
</head>
<body>
    <h1>نصب Llama 3.1</h1>
    <p>برای نصب Llama 3.1، مراحل زیر را دنبال کنید:</p>
    <p>ابتدا باید برنامه Llama 3.1 را دانلود و نصب کنید. می‌توانید از <a href="https://ollama.com/" target="_blank">لینک رسمی </a> دانلود استفاده کنید. پس از دانلود، مراحل نصب را طبق دستورالعمل‌های ارائه شده دنبال کنید.</p>
    <img src="images/1.png" alt="نصب برنامه Llama 3.1" />
    <p>پس از نصب برنامه، باید <a href="https://ollama.com/library/llama3.1" target="_blank"> مدل مورد نظر </a> نظر خود را انتخاب کنید. Llama 3.1 چندین مدل مختلف ارائه می‌دهد که می‌توانید بر اساس نیاز خود یکی از آن‌ها را انتخاب کنید. برای این کار، به منوی مدل‌ها در برنامه بروید و مدل مناسب را انتخاب کنید.</p>
    <img src="images/2.png" alt="انتخاب مدل Llama 3.1" />
    <p>پس از انتخاب مدل، کد مورد نظر را کپی و در ترمینال آن را اجرا کنید.</p>
    <img src="images/3.png" alt="تنظیمات نهایی Llama 3.1" />
    <p>پس از انجام این مراحل، Llama 3.1 آماده استفاده است. شما می‌توانید از آن برای پروژه‌های خود بهره‌برداری کنید.</p>
</body>
</html>

In [1]:
import os
import re
from pypdf import PdfReader
from langchain import hub
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_ollama import OllamaLLM, OllamaEmbeddings



<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB; /* رنگ پس‌زمینه بلوک‌های کد */
            padding: 10px;
            border-radius: 5px;
            direction: ltr; /* جهت متنی کدها از چپ به راست */
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 14px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul, ol {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        ul ul, ul ol, ol ul, ol ol {
            padding-right: 20px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b> تنظیم و بارگذاری مدل‌های Ollama (LLM و Embeddings)</b>
        <ul>
            <li><b>هدف:</b> تنظیم و بارگذاری مدل زبانی بزرگ (LLM) و امبدینگ‌های Ollama برای استفاده در پردازش‌های بعدی.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>llm_model</code>: نام مدل زبانی مورد استفاده، در اینجا "llama3.1".</li>
                    <li><code>llm_url</code>: آدرس URL برای دسترسی به مدل زبانی، معمولاً به صورت محلی.</li>
                    <li><code>OllamaLLM</code>: کلاس یا تابعی که برای بارگذاری مدل زبانی Ollama استفاده می‌شود.</li>
                    <li><code>OllamaEmbeddings</code>: کلاس یا تابعی که برای تنظیم امبدینگ‌های Ollama استفاده می‌شود.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>تعیین نام مدل و URL مربوط به مدل زبانی.</li>
                    <li>ایجاد نمونه‌ای از مدل زبانی با استفاده از <code>OllamaLLM</code> و تنظیم URL پایه.</li>
                    <li>تنظیم امبدینگ‌های Ollama با استفاده از <code>OllamaEmbeddings</code> و همان نام مدل و URL پایه.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [2]:
llm_model= "llama3.1"
llm_url = "http://127.0.0.1:11434"

#Loading The LLM (Language Model)
llm = OllamaLLM(model=llm_model, base_url=llm_url)

#Setting Ollama Embeddings
# http://localhost:11434/api/tags
embed_model = OllamaEmbeddings(model=llm_model,base_url=llm_url)



In [3]:
#Loading Text

with open('text.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

In [4]:
file_content

'هادی پاکزاد\nنام تولد\tهادی پاکزاد\nزاده\t۳ دی ۱۳۶۱\nمشهد، ایران\nدرگذشته\t۶ خرداد ۱۳۹۵ (۳۳ سال)\nخودکشی با قرص\u200cهای آرام\u200cبخش و مسکن\nمشهد، ایران\nژانر موسیقی موج نو آلترنتیو راک سایکدلیک راک، موسیقی راک\nساز(ها)\tکیبورد، گیتار الکتریک، درامز\nسال\u200cهای فعالیت\t۱۳۸۱ – ۱۳۹۵\nوبگاه\t\nwww.HadiPakzad.com\nهادی پاکزاد (زادهٔ ۳ دی ۱۳۶۱ مشهد – درگذشتهٔ ۶ خرداد ۱۳۹۵ مشهد) آهنگ\u200cساز، نویسنده، خواننده و ترانه\u200cسرای موسیقی آلترنتیو راک۶ بود.\n\nفعالیت هنری\n\nاو از سال ۱۳۸۱ فعالیت\u200cهای خود را در حوزهٔ موسیقی آغاز کرد و نخستین آلبوم رسمی\u200cاش به نام تاریکی دی ماه سال ۱۳۹۴ روانه بازار موسیقی کشور شد. هادی پاکزاد از خواننده\u200cهای راک و آلترناتیو[۲] در ایران است. قبل از این که با موسیقی درگیر شود به اشعار کلاسیک و نوشته\u200cهای فلسفی علاقه\u200cمند بود. در سال ۲۰۰۰، بعد از کسب تجربهٔ یادگیری پیانو و گیتار، در یک استودیوی خانگی و با کمک موسیقیدان\u200cهای محلی شروع به تولید اولین آلبومش می\u200cکند. تمام اشعار آهنگ\u200cهای این آلبوم توسط خودش سروده شده و به خاطر نبود 

In [5]:
# # Loading PDF Document and split

# def load_pdf(file_path):
#     reader = PdfReader(file_path)
#     text = ""
#     for page in reader.pages:
#         text += page.extract_text()
#     return text

# def split_text(text: str):
#     split_text = re.split('\n \n', text)
#     return [i for i in split_text if i != ""]


In [6]:
# file_content = load_pdf(file_path="ebk.pdf")
# len(file_content)

In [7]:
# import re

# def clean_text(text):
#     # حذف کاراکترهای نامعتبر و تبدیل به UTF-8
#     return text.encode('utf-8', 'ignore').decode('utf-8')

# def remove_special_characters(text):
#     # حذف کاراکترهای غیر ASCII و کاراکتر NULL
#     text = re.sub(r'[^\x20-\x7E]+', '', text)  # حذف کاراکترهای غیر قابل چاپ و کاراکتر NULL
#     text = re.sub(r'\s+', ' ', text)  # کاهش فاصله‌های خالی متوالی به یک فاصله
#     return text.strip()  # حذف فاصله‌های خالی در ابتدای و انتهای متن

# # مرحله 1: تمیز کردن متن
# file_content = clean_text(file_content)
# file_content = remove_special_characters(file_content)
# len(file_content)

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b> تقسیم‌بندی متن به بخش‌های کوچک</b>
        <ul>
            <li><b>هدف:</b> تقسیم متن بزرگ به بخش‌های کوچک‌تر برای پردازش بهتر.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>RecursiveCharacterTextSplitter</code>: کلاس برای تقسیم متن به بخش‌های کوچکتر.</li>
                    <li><code>chunk_size=1500</code>: حداکثر تعداد کاراکترها در هر بخش.</li>
                    <li><code>chunk_overlap=150</code>: تعداد کاراکترهای تکراری بین بخش‌ها.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ایجاد نمونه از <code>RecursiveCharacterTextSplitter</code>.</li>
                    <li>تقسیم متن با استفاده از متد <code>split_text</code>.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [8]:
# Splitting Text or PDF  into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
chunks = text_splitter.split_text(file_content)


<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b> ایجاد Vector Store با استفاده از Chroma</b>
        <ul>
            <li><b>هدف:</b> ذخیره‌سازی برداری از بخش‌های متنی برای جستجو و بازیابی سریع‌تر.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>Chroma</code>: سیستم مدیریت بردار برای ذخیره‌سازی و بازیابی بردارهای متنی.</li>
                    <li><code>from_texts</code>: متد تبدیل متون به بردارهای عددی.</li>
                    <li><code>chunks</code>: لیست بخش‌های متنی تقسیم‌بندی شده.</li>
                    <li><code>embed_model</code>: مدل امبدینگ برای تبدیل متن به بردار.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>تبدیل هر بخش از <code>chunks</code> به بردار با استفاده از <code>embed_model</code>.</li>
                    <li>ذخیره بردارها در <code>Chroma</code> برای جستجو و بازیابی سریع.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [10]:
# Creating Vector Store using Chroma
vector_store = Chroma.from_texts(chunks, embed_model)

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b> ایجاد Retriever</b>
        <ul>
            <li><b>هدف:</b> فراهم کردن امکان بازیابی بخش‌های مرتبط از بردارهای ذخیره شده.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>vector_store</code>: پایگاه داده برداری.</li>
                    <li><code>as_retriever()</code>: متد ایجاد بازیاب از <code>vector_store</code>.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ایجاد شیء بازیاب با استفاده از <code>as_retriever()</code>.</li>
                    <li>استفاده از بازیاب برای یافتن بخش‌های مرتبط بر اساس پرسش.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [11]:
# Creating Retriever
retriever = vector_store.as_retriever()

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b> ایجاد Retrieval Chain با استفاده از LLM و Retriever</b>
        <ul>
            <li><b>هدف:</b> ایجاد زنجیره پردازشی که از مدل زبانی بزرگ و بازیاب استفاده کند.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>create_retrieval_chain</code>: تابع ایجاد زنجیره پردازشی.</li>
                    <li><code>combine_docs_chain=llm</code>: استفاده از مدل زبانی برای پردازش اسناد.</li>
                    <li><code>retriever</code>: بازیاب اطلاعات.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ترکیب قابلیت‌های بازیاب و مدل زبانی در یک زنجیره.</li>
                    <li>استفاده از زنجیره برای تولید پاسخ‌های دقیق‌تر به پرسش‌های کاربران.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [12]:
# Creating Retrieval Chain using LLM and Retriever
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b> Retrieval-QA Chat Prompt</b>
        <ul>
            <li><b>هدف:</b> بارگذاری پرامپت چت سوال و جواب از مخزن LangChain.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>hub.pull</code>: متد دانلود منابع از مخزن.</li>
                    <li><code>"langchain-ai/retrieval-qa-chat"</code>: مسیر مخزن پرامپت.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>دانلود پرامپت با استفاده از <code>hub.pull</code>.</li>
                    <li>استفاده از پرامپت در مدل زبانی برای تعاملات سوال و جواب.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [13]:
# Retrieval-QA Chat Prompt
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

/Users/majidjafari/Documents/proposal/code/AhmadRAG_env/lib/python3.11/site-packages/langsmith/client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b> Combining Documents</b>
        <ul>
            <li><b>هدف:</b> ترکیب اسناد بازیابی شده برای ایجاد پاسخ‌های دقیق‌تر.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>create_stuff_documents_chain</code>: تابع ایجاد زنجیره ترکیب اسناد.</li>
                    <li><code>llm</code>: مدل زبانی بزرگ.</li>
                    <li><code>retrieval_qa_chat_prompt</code>: پرامپت برای تعاملات QA Chat.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ایجاد زنجیره ترکیب اسناد با استفاده از <code>llm</code> و پرامپت.</li>
                    <li>پردازش اسناد بازیابی شده برای تولید پاسخ نهایی.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [14]:
# Combining Documents
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 1px;
            border-radius: 1px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 1px 1px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 10px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b> Final Retrieval Chain</b>
        <ul>
            <li><b>هدف:</b> ایجاد زنجیره نهایی بازیابی برای پاسخ‌های دقیق‌تر.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>create_retrieval_chain</code>: تابع ایجاد زنجیره نهایی.</li>
                    <li><code>retriever</code>: بازیاب اطلاعات.</li>
                    <li><code>combine_docs_chain</code>: زنجیره ترکیب اسناد.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ایجاد زنجیره نهایی با ترکیب <code>retriever</code> و <code>combine_docs_chain</code>.</li>
                    <li>بازیابی بخش‌های مرتبط و تولید پاسخ نهایی با استفاده از زنجیره.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [15]:
#Final Retrieval Chain
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)    


In [16]:
query1 = "هادی پاکزاد کیست"
query2 ="هادی پاکزاد در چه زمینه ای فعالیت می کند"
query3 ="علت مرگ هادی پاکزاد چیست و چه سالی اتفاق افتاد"

In [17]:
response1 = llm.invoke(query1)
response2 = llm.invoke(query2)
response3 = llm.invoke(query3)

In [22]:
print("_"*50)
print(query1 , "? \n",response1)
print("_"*50)
print(query2 , "? \n",response2)
print("_"*50)
print(query3 , "? \n",response3)

__________________________________________________
هادی پاکزاد کیست ? 
 هادی پاکزاد (زاده ۱۳۳۵ تهران) ، شاعر، نویسنده و مترجم ایرانی است. از او به عنوان "شاعری که همگی شعر را می فهمند" صحبت کرده اند.
هادی پاک زاد در سال ۱۳۳۵ خورشیدی در محلهٔ رضویتپه (میدان قدس) تهران به دنیا آمد. وی فارغ‌التحصیل رشتهٔ حقوق دانشگاه ملی است.او از بنیان‌گذاران کانون شعر ایران و نویسندهٔ نخستین اثر منظوم دربارهٔ ادبیات معاصر ایران با عنوان «شعر نو» می‌باشد.
__________________________________________________
هادی پاکزاد در چه زمینه ای فعالیت می کند ? 
 هادی پاکزاد، بازیگر سینما و تئاتر است.
__________________________________________________
علت مرگ هادی پاکزاد چیست و چه سالی اتفاق افتاد ? 
 به دلیل سوء غذا خوردن به سرطان کبد در سن ۷۵ سالگی درگذشت.


In [19]:
RAGresponse1 = retrieval_chain.invoke({"input": query1})
RAGresponse2 = retrieval_chain.invoke({"input": query2})
RAGresponse3 = retrieval_chain.invoke({"input": query3})


In [23]:
print("_"*50)
print(query1 , "? \n",RAGresponse1['answer'])
print("_"*50)
print(query2 , "? \n",RAGresponse2['answer'])
print("_"*50)
print(query3 , "? \n",RAGresponse3['answer'])

__________________________________________________
هادی پاکزاد کیست ? 
 هادی پاکزاد خواننده و ترانه‌سرای ایرانی است که تا پیش از درگذشت ۳۳ساله خود به عنوان یکی از هنرمندان آلترناتیو شناخته می‌شد.
__________________________________________________
هادی پاکزاد در چه زمینه ای فعالیت می کند ? 
 هادی پاکزاد هنرمند موسیقی است و آلبوم های متعددی در زمینه موسیقی آلترناتیو داشته است.
__________________________________________________
علت مرگ هادی پاکزاد چیست و چه سالی اتفاق افتاد ? 
 درگذشت هادی پاکزاد در روز ۶ خرداد ۱۳۹۵ (۲۵ ژوئن ۲۰۱۶) و علت مرگ او را خودکشی اعلام شده.


<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>گزارش درباره هادی پاکزاد</title>
    <style>
        body {
            direction: rtl;
            line-height: 1.6;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        h1, h2 {
            color: #2980b9;
        }
        pre {
            background-color: #FFC0CB; /* رنگ پس‌زمینه بلوک‌های کد */
            padding: 10px;
            border-radius: 5px;
            direction: ltr; /* جهت متنی کدها از چپ به راست */
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 14px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        strong {
            font-weight: bold;
        }
        .incorrect {
            background-color: #f8d7da; /* رنگ پس‌زمینه برای پاسخ‌های نادرست */
            padding: 10px;
            border: 1px solid #f5c6cb;
            border-radius: 5px;
            margin-bottom: 20px;
        }
        .correct {
            background-color: #d4edda; /* رنگ پس‌زمینه برای پاسخ‌های درست */
            padding: 10px;
            border: 1px solid #c3e6cb;
            border-radius: 5px;
            margin-bottom: 20px;
        }
    </style>
</head>
<body>
    <h1>گزارش درباره هادی پاکزاد</h1>
    <h2>۱. هادی پاکزاد کیست؟</h2>
    <div class="incorrect">
        <strong>پاسخ نادرست:</strong>
        <p>هادی پاکزاد (زاده ۱۳۳۵ تهران)، شاعر، نویسنده و مترجم ایرانی است. از او به عنوان "شاعری که همگی شعر را می فهمند" صحبت کرده‌اند.</p>
    </div>
    <div class="correct">
        <strong>پاسخ درست:</strong>
        <p>هادی پاکزاد خواننده و ترانه‌سرای ایرانی است که تا پیش از درگذشت ۳۳ ساله خود به عنوان یکی از هنرمندان آلترناتیو شناخته می‌شد.</p>
    </div>
    <h2>۲. هادی پاکزاد در چه زمینه‌ای فعالیت می‌کند؟</h2>
    <div class="incorrect">
        <strong>پاسخ نادرست:</strong>
        <p>هادی پاکزاد، بازیگر سینما و تئاتر است.</p>
    </div>
    <div class="correct">
        <strong>پاسخ درست:</strong>
        <p>هادی پاکزاد هنرمند موسیقی است و آلبوم‌های متعددی در زمینه موسیقی آلترناتیو داشته است.</p>
    </div>
    <h2>۳. علت مرگ هادی پاکزاد چیست و چه سالی اتفاق افتاد؟</h2>
    <div class="incorrect">
        <strong>پاسخ نادرست:</strong>
        <p>به دلیل سوء غذا خوردن به سرطان کبد در سن ۷۵ سالگی درگذشت.</p>
    </div>
    <div class="correct">
        <strong>پاسخ درست:</strong>
        <p>درگذشت هادی پاکزاد در روز ۶ خرداد ۱۳۹۵ (۲۵ ژوئن ۲۰۱۶) و علت مرگ او را خودکشی اعلام شده است.</p>
    </div>
    <h2>۴. نکته مهم</h2>
    <p>ضمناً امسال ۱۴۰۳ است و جمع ۱۳۳۵، ۷۵ می‌شود ۱۴۱۰.</p>
</body>
</html>